In [ ]:
import os
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import hist
from hist import Hist
import os, sys
import re
import uproot
import pickle
import vector
import mplhep as hep
import math
vector.register_awkward()

# parameters for matplotlib styling
plt.style.use(hep.style.CMS)
matplotlib.rcParams.update({'figure.max_open_warning': 0})
matplotlib.rcParams.update({'font.size': 20})

In [ ]:
# parameters
tag = '2018_MC'
unblind = False                               # don't show D_exp for SUEP
show = 'all'                                  # set to 'all' or list of plots or plot name
save_to_file = ['ABCDvars_2D_mult']                      # set to 'all' or list of plots or plot name
output_label = 'SUEP'

datasets = [
           "QCD_HT1000to1500_TuneCP5_13TeV-madgraphMLM-pythia8+RunIIAutumn18MiniAOD-102X_upgrade2018_realistic_v15-v1+MINIAODSIM",
           "QCD_HT100to200_TuneCP5_13TeV-madgraphMLM-pythia8+RunIIAutumn18MiniAOD-102X_upgrade2018_realistic_v15-v1+MINIAODSIM",
           "QCD_HT1500to2000_TuneCP5_13TeV-madgraphMLM-pythia8+RunIIAutumn18MiniAOD-102X_upgrade2018_realistic_v15-v1+MINIAODSIM",
           "QCD_HT2000toInf_TuneCP5_13TeV-madgraphMLM-pythia8+RunIIAutumn18MiniAOD-102X_upgrade2018_realistic_v15-v1+MINIAODSIM",
           "QCD_HT200to300_TuneCP5_13TeV-madgraphMLM-pythia8+RunIIAutumn18MiniAOD-102X_upgrade2018_realistic_v15-v1+MINIAODSIM",
           "QCD_HT300to500_TuneCP5_13TeV-madgraphMLM-pythia8+RunIIAutumn18MiniAOD-102X_upgrade2018_realistic_v15-v1+MINIAODSIM",
           "QCD_HT500to700_TuneCP5_13TeV-madgraphMLM-pythia8+RunIIAutumn18MiniAOD-102X_upgrade2018_realistic_v15-v1+MINIAODSIM",
           "QCD_HT700to1000_TuneCP5_13TeV-madgraphMLM-pythia8+RunIIAutumn18MiniAOD-102X_upgrade2018_realistic_v15-v1+MINIAODSIM",
           "SUEP-m1000-darkPho+RunIIAutumn18-private+MINIAODSIM",
           "SUEP-m1000-darkPhoHad+RunIIAutumn18-private+MINIAODSIM",
           "SUEP-m1000-generic+RunIIAutumn18-private+MINIAODSIM",
           "SUEP-m125-darkPho+RunIIAutumn18-private+MINIAODSIM",
           "SUEP-m125-darkPhoHad+RunIIAutumn18-private+MINIAODSIM",
           "SUEP-m125-generic+RunIIAutumn18-private+MINIAODSIM",
           "SUEP-m400-darkPho+RunIIAutumn18-private+MINIAODSIM",
           "SUEP-m400-darkPhoHad+RunIIAutumn18-private+MINIAODSIM",
           "SUEP-m400-generic+RunIIAutumn18-private+MINIAODSIM",
           "SUEP-m750-darkPho+RunIIAutumn18-private+MINIAODSIM",
           "SUEP-m750-darkPhoHad+RunIIAutumn18-private+MINIAODSIM",
           "SUEP-m750-generic+RunIIAutumn18-private+MINIAODSIM"
           ]
plotDir = os.environ['HOME']+"/SUEPCoffea_dask/plotting/"
infile_names = [plotDir+dataset+'_ABCD_plot.pkl' for dataset in datasets]
dataDir = "/work/submit/{0}/SUEP/{1}".format(os.environ['USER'],tag)

In [ ]:
# load files from pickle
def openpkl(infile_name):
    with (open(infile_name, "rb")) as openfile:
        plots = pickle.load(openfile)
        plots.update(pickle.load(openfile))
    return plots

In [ ]:
SUEP_plots, QCD_plots = {}, {}
for dataset, infile_name in zip(datasets, infile_names):
    if not os.path.isfile(infile_name): 
        print("WARNING:",dataset,"doesn't exist")
        continue
    print("Loading",dataset)
    if 'SUEP' in dataset:
        SUEP_plots[dataset] = openpkl(infile_name)
    if 'QCD' in dataset:
        QCD_plots[dataset] = openpkl(infile_name)

In [ ]:
def plot1d(h, ax, label):
    values, x = h.to_numpy()
    if 'QCD' in label: 
        ax.step(x[:-1],values, label=label, color='midnightblue')
        ax.fill_between(x[:-1],values,step='pre', color='midnightblue')
    else:
        ax.step(x[:-1],values, label=label)
    ax.set_xlabel(h.axes[0].label)
    ax.set_ylabel("Events")
    
def plot2d(h, ax, log=False):
    w, x, y = h.to_numpy()
    if log: mesh = ax.pcolormesh(x, y, w.T, cmap="RdYlBu", norm=matplotlib.colors.LogNorm())
    else: mesh = ax.pcolormesh(x, y, w.T, cmap="RdYlBu")
    ax.set_xlabel(h.axes[0].label)
    ax.set_ylabel(h.axes[1].label)
    fig.colorbar(mesh)

In [ ]:
# define a list of plots to show
if type(show) == str and show == 'all': 
    showlist = list(SUEP_plots[list(SUEP_plots.keys())[0]].keys())
elif type(show) == str and show in SUEP_plots[list(SUEP_plots.keys())[0]].keys():
    showlist = [show]
elif type(show) == list: 
    showlist = show
print("Showing plots:",showlist)

# define a savelist of plots to save to file
if type(save_to_file) == str and save_to_file == 'all': 
    savelist = list(plots.keys())
elif type(save_to_file) == str and save_to_file in plots.keys():
    savelist = [save_to_file]
elif type(save_to_file) == list: 
    savelist = save_to_file
print("Saving plots:",savelist)

In [ ]:
def find_nth(string, substring, n):
    if (n == 1):
        return string.find(substring)
    else:
        return string.find(substring, find_nth(string, substring, n - 1) + 1)

In [ ]:
#### 1D PLOTS

for plot_label in showlist:
    #if 'D_obs' in plot_label and (not unblind): continue
    if '2D' in plot_label: continue
    save = plot_label in savelist
        
    # define figure, axis on which to plot QCD and SUEP
    fig, ax = plt.subplots(figsize=(8,5))
    hep.cms.label(data=False,year='2018', fontsize=20)
    
    # keep the SUEP hists separate for each sample
    SUEP_hists,SUEP_labels = [],[]
    for sample in list(SUEP_plots.keys()):
        i = find_nth(sample,'+',1)
        SUEP_labels.append(sample[:i])
        SUEP_hists.append(SUEP_plots[sample][plot_label])
    
    # combine the QCD hists
    for i,QCD_sample in enumerate(QCD_plots.keys()):
        if i==0: hQCD = QCD_plots[QCD_sample][plot_label]
        else: hQCD = hQCD + QCD_plots[QCD_sample][plot_label]
    
    # plot QCD, SUEP
    for hSUEP, l in zip(SUEP_hists, SUEP_labels): plot1d(hSUEP, ax, l)
    plot1d(hQCD, ax, 'QCD')
    
    ax.set_yscale('log')
    ax.legend(loc='best', fontsize=10, framealpha=1, facecolor='white', shadow=True)
    fig.tight_layout()
    fig.show()
    
    if save: fig.savefig(output_label + "_" + plot_label +".pdf", bbox_inches='tight')

In [ ]:
### 2D PLOTS
datasets2d = ['QCD',
            "SUEP-m1000-darkPho+RunIIAutumn18-private+MINIAODSIM",
           "SUEP-m125-darkPho+RunIIAutumn18-private+MINIAODSIM",
           "SUEP-m400-darkPho+RunIIAutumn18-private+MINIAODSIM",
           "SUEP-m750-darkPho+RunIIAutumn18-private+MINIAODSIM"]

for dataset in datasets2d:
    for plot_label in showlist:
        if 'D_obs' in plot_label and (not unblind): continue
        if '2D' not in plot_label: continue
        save = plot_label in savelist

        # define figure, axis on which to plot QCD and SUEP
        fig, ax = plt.subplots(figsize=(8,7))
        hep.cms.label(data=False,year='2018', fontsize=20)

        if 'SUEP' in dataset: h = SUEP_plots[dataset][plot_label]
        elif 'QCD' in dataset and 'QCD' != dataset: h = QCD_plots[dataset][plot_label]
        elif 'QCD' == dataset: 
            for i,QCD_sample in enumerate(QCD_plots.keys()):
                if i==0: h = QCD_plots[QCD_sample][plot_label]
                else: h = h + QCD_plots[QCD_sample][plot_label]
        if '+' in dataset:
            i = find_nth(dataset,'+',1)
            l = dataset[:i]
        else: l = dataset
        plot2d(h, ax, log=True)

        fig.suptitle(l, y=0.95)
        fig.tight_layout()
        fig.show()

        if save: fig.savefig(l + "_" + plot_label +".pdf", bbox_inches='tight')

In [ ]:
selection = 'mult'

# combine the QCD hists
for i,QCD_sample in enumerate(QCD_plots.keys()):
    if i==0: 
        hQCD_Dexp = QCD_plots[QCD_sample]['D_exp_'+selection]
        hQCD_Dexp_errs = QCD_plots[QCD_sample]['D_exp_'+selection].variances()
        hQCD_Dobs = QCD_plots[QCD_sample]['D_obs_'+selection]
        hQCD_Dobs_errs = QCD_plots[QCD_sample]['D_obs_'+selection].variances()
    else: 
        hQCD_Dexp = hQCD_Dexp + QCD_plots[QCD_sample]['D_exp_'+selection]
        hQCD_Dexp_errs = hQCD_Dexp_errs + QCD_plots[QCD_sample]['D_exp_'+selection].variances()
        hQCD_Dobs = hQCD_Dobs + QCD_plots[QCD_sample]['D_obs_'+selection]
        hQCD_Dobs_errs = hQCD_Dobs_errs + QCD_plots[QCD_sample]['D_obs_'+selection].variances()
        
hQCD_Dobs_errs = np.sqrt(hQCD_Dobs_errs)
hQCD_Dexp_errs = np.sqrt(hQCD_Dexp_errs)

#Set up variables for the stacked histogram
plt.figure(figsize=(12,10))
plt.gcf().subplots_adjust(bottom=0.15, left=0.17)
ax1 = plt.subplot2grid((4,1), (0,0),rowspan=2)

Dexp, x = hQCD_Dexp.to_numpy()
ax1.step(x[:-1],Dexp, color='maroon',label=r"$D_{exp}$", where='mid')
ax1.errorbar(x[:-1],Dexp, yerr=hQCD_Dexp_errs, color="maroon".upper(), fmt='o')

Dobs, x = hQCD_Dobs.to_numpy()
ax1.step(x[:-1],Dobs, color='blue',label=r"$D_{obs}$", where= 'mid')
ax1.errorbar(x[:-1],Dobs, yerr=hQCD_Dobs_errs, color="blue".upper(), fmt='o')

xmin = 0.6
ax1.set_yscale("log")
ax1.set_ylim([0.001, 1000])
ax1.set_xlim([xmin,1.0])
    
#Set parameters that will be used to make the plots prettier
#max_y = ax1.gca().get_ylim()[1]
max_y = 10000
max_x = max(x[:-1])
min_x = xmin
x_range = max_x - min_x
lower_label = min_x - x_range*0.05
upper_label = max_x - x_range*0.35
    
#X and Y labels (Do not use the central matplotlob default)
ax1.set_xlabel('D region', x=1, ha='right', fontsize=15)
ax1.set_ylabel("Events/bin", y=1, ha='right')
    
cms = ax1.text(
        lower_label, max_y*1.08, u"CMS $\it{preliminary}$",
        fontsize=16, fontweight='bold',
)

lumi = ax1.text(
        upper_label, max_y*1.08, r"%.1f fb$^{-1}$ (13 TeV)" % 60,
        fontsize=14, 
)
ax1.legend()

ax2 = plt.subplot2grid((4,1), (2,0),sharex=ax1)
plt.setp(ax1.get_xticklabels(), visible=False)
yerrors = [Dobs/Dexp - (Dobs-hQCD_Dobs_errs)/(Dexp+hQCD_Dexp_errs),(Dobs+hQCD_Dobs_errs)/(Dexp-hQCD_Dexp_errs) - Dobs/Dexp]
#yerrors = [(QCD_obs-QCD_obs_errors)/(QCD_exp+QCD_exp_errors),(QCD_obs+QCD_obs_errors)/(QCD_exp-QCD_exp_errors)]
ax2.errorbar(x[:-1],np.where(Dobs>0,Dobs/Dexp,1),yerr=yerrors, color="black", fmt='o')
ax2.axhline(1, ls="--")
ax2.set_ylim(0.1,5)
ax2.set_xlabel("D region", fontsize=15)
plt.savefig("D_region_mult.pdf")

In [ ]:
from collections import defaultdict

def nested_dict(n, type):
    if n == 1:
        return defaultdict(type)
    else:
        return defaultdict(lambda: nested_dict(n-1, type))
    
SUEPdata = nested_dict(3,float)
QCDdata = nested_dict(2,float)

In [ ]:
for plot_label in ['A_'+selection,'B_'+selection,'C_'+selection,'D_exp_'+selection,'D_obs_'+selection]:
    save = True
        
    # define figure, axis on which to plot QCD and SUEP
    fig, ax = plt.subplots(figsize=(8,5))
    hep.cms.label(data=False,year='2018', fontsize=20)
    
    # keep the SUEP hists separate for each sample
    SUEP_hists,SUEP_labels = [],[]
    for sample in list(SUEP_plots.keys()):
        i = find_nth(sample,'+',1)
        SUEP_labels.append(sample[:i])
        SUEP_hists.append(SUEP_plots[sample][plot_label])
        
        SUEPdata[sample[:i]][plot_label]["value"] = np.sum(SUEP_plots[sample][plot_label].values())
        SUEPdata[sample[:i]][plot_label]["err"] = np.sum(SUEP_plots[sample][plot_label].variances())
    
    # combine the QCD hists
    for i,QCD_sample in enumerate(QCD_plots.keys()):
        if i==0: hQCD = QCD_plots[QCD_sample][plot_label]
        else: hQCD = hQCD + QCD_plots[QCD_sample][plot_label]
    
    # plot QCD, SUEP
    for hSUEP, l in zip(SUEP_hists, SUEP_labels): plot1d(hSUEP, ax, l)
    plot1d(hQCD, ax, 'QCD')
    SUEPdata['QCD'][plot_label]["value"] = np.sum(hQCD.values())
    SUEPdata['QCD'][plot_label]["err"] = np.sum(hQCD.variances())
    
    ax.set_yscale('log')
    ax.legend(loc='best', fontsize=10, framealpha=1, facecolor='white', shadow=True)
    fig.tight_layout()
    fig.show()
    
    if save: fig.savefig(output_label + "_" + plot_label +".pdf", bbox_inches='tight')

In [ ]:
table = []
for i in range(len(list(SUEPdata.keys()))):
    label = list(SUEPdata.keys())[i]
    line = []
    line.append(label)
    for r in ['A_'+selection,'B_'+selection,'C_'+selection,'D_exp_'+selection,'D_obs_'+selection]:
        line.append("{:0.2f} +/- {:0.2f}".format(SUEPdata[label][r]['value'],SUEPdata[label][r]['err']))
    if 'QCD' in label:
        line.append((SUEPdata[label]['D_exp_'+selection]['value']-SUEPdata[label]['D_obs_'+selection]['value'])/SUEPdata[label]['D_exp_'+selection]['value'])
        line.append(0)
    else:
        line.append(0)
        line.append(SUEPdata[label]['D_obs_'+selection]['value']/math.sqrt(SUEPdata[label]['D_obs_'+selection]['value']+SUEPdata[label]['D_exp_'+selection]['value']+SUEPdata[label]['D_obs_'+selection]['err']+SUEPdata['QCD']['D_exp_'+selection]['err']))
    table.append(line)                                                         

In [ ]:
from tabulate import tabulate
t = tabulate(table,
               headers=['Process', 'A region', 'B region', 'C region', 'D expected', 'D observed', '(exp-obs)/exp', 'S/\u221A(S+B+\u03C3S+\u03C3B)'])
with open('table.txt', 'w') as f:
    f.write(t)
t

In [ ]:
from tabulate import tabulate
print(tabulate(
    [['QCD', "{:0.2f} +/- {:0.2f}".format(SUEPdata['QCD']['A_mult']['value'],A_QCD_errors),"{:0.2f} +/- {:0.2f}".format(B_QCD,B_QCD_errors),"{:0.2f} +/- {:0.2f}".format(C_QCD,C_QCD_errors),"{:0.2f} +/- {:0.2f}".format(D_exp_QCD,D_exp_QCD_errors),"{:0.2f} +/- {:0.2f}".format(D_obs_QCD,D_obs_QCD_errors),(D_exp_QCD-D_obs_QCD)/D_exp_QCD,0], 
     ['SUEP m125', "{:0.2f} +/- {:0.2f}".format(A_SUEP1,A_SUEP1_errors),"{:0.2f} +/- {:0.2f}".format(B_SUEP1,B_SUEP1_errors),"{:0.2f} +/- {:0.2f}".format(C_SUEP1,C_SUEP1_errors),"{:0.2f} +/- {:0.2f}".format(D_exp_SUEP1,D_exp_SUEP1_errors),"{:0.2f} +/- {:0.2f}".format(D_obs_SUEP1,D_obs_SUEP1_errors),0,D_obs_SUEP1/math.sqrt(D_obs_SUEP1+D_exp_QCD+D_obs_SUEP1_errors+D_exp_QCD_errors)],
     ['SUEP m400', "{:0.2f} +/- {:0.2f}".format(A_SUEP2,A_SUEP2_errors),"{:0.2f} +/- {:0.2f}".format(B_SUEP2,B_SUEP2_errors),"{:0.2f} +/- {:0.2f}".format(C_SUEP2,C_SUEP2_errors),"{:0.2f} +/- {:0.2f}".format(D_exp_SUEP2,D_exp_SUEP2_errors),"{:0.2f} +/- {:0.2f}".format(D_obs_SUEP2,D_obs_SUEP2_errors),0,D_obs_SUEP2/math.sqrt(D_obs_SUEP2+D_exp_QCD+D_obs_SUEP2_errors+D_exp_QCD_errors)],
     ['SUEP m750', "{:0.2f} +/- {:0.2f}".format(A_SUEP3,A_SUEP3_errors),"{:0.2f} +/- {:0.2f}".format(B_SUEP3,B_SUEP3_errors),"{:0.2f} +/- {:0.2f}".format(C_SUEP3,C_SUEP3_errors),"{:0.2f} +/- {:0.2f}".format(D_exp_SUEP3,D_exp_SUEP3_errors),"{:0.2f} +/- {:0.2f}".format(D_obs_SUEP3,D_obs_SUEP3_errors),0,D_obs_SUEP3/math.sqrt(D_obs_SUEP3+D_exp_QCD+D_obs_SUEP3_errors+D_exp_QCD_errors)],
     ['SUEP m1000', "{:0.2f} +/- {:0.2f}".format(A_SUEP4,A_SUEP4_errors),"{:0.2f} +/- {:0.2f}".format(B_SUEP4,B_SUEP4_errors),"{:0.2f} +/- {:0.2f}".format(C_SUEP4,C_SUEP4_errors),"{:0.2f} +/- {:0.2f}".format(D_exp_SUEP4,D_exp_SUEP4_errors),"{:0.2f} +/- {:0.2f}".format(D_obs_SUEP4,D_obs_SUEP4_errors),0,D_obs_SUEP4/math.sqrt(D_obs_SUEP4+D_exp_QCD+D_obs_SUEP4_errors+D_exp_QCD_errors)],
    ], 
    headers=['Process', 'A region', 'B region', 'C region', 'D expected', 'D observed', '(exp-obs)/exp', 'S/\u221A(S+B+\u03C3S+\u03C3B)']))